1. Rainbow and Ape-X Expiriments 
    1. We release a set of hyper parameters for CartPole-v1 and Classic Control and Atari
    2. We release code for Rainbow that can train X steps in Y minutes on a Mac M2 Chip
    3. We also release a version of Ape-X as described in the original paper, and an Ape-X with rainbow
        1. Compare results of each 
        2. Compare Ape-X with different Rainbow components added or removed
    4. We compare the different models of DQN as seen in their papers to rainbow, the different individual components to rainbow, and rainbow with individual components removed
    10. Compare rainbow training speeds with different levels of numerical precision and datatypes
        1. Mixed precision using torch.amp 
        2. Lower matmul precision
            1. comparing medium, high, and highest 
            2. https://pytorch.org/docs/master/generated/torch.set_float32_matmul_precision.html?highlight=precision#torch.set_float32_matmul_precision
    11. Ape-X Hyper parameter sweep and sensitivities
    12. Exploration methods for Rainbow Ape-X
        1. Just noisy nets (same for all actors)
        2. Noisy nets and varying epsilon 
        3. Adding a constant that changes variance of noisy nets for action selection
        4. AlphaStar Agents

## Rainbow on CartPole-v1
Hyperparameters are based on the hyperopt experiments, quantized trial 27 with some minor changes

In [1]:
import gymnasium as gym
import sys

from utils import CategoricalCrossentropy, KLDivergence
sys.path.append('../..')
from dqn.rainbow.rainbow_agent import RainbowAgent
from agent_configs import RainbowConfig
from game_configs import CartPoleConfig

# env = ClipReward(AtariPreprocessing(gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array"), terminal_on_life_loss=True), -1, 1) # as recommended by the original paper, should already include max pooling
# env = FrameStack(env, 4)
env = gym.make("CartPole-v1")


config_dict = {
  "dense_layers_widths": [128, 128],
  "value_hidden_layers_widths": [64, 64],
  "advatage_hidden_layers_widths": [64, 64],
  "adam_epsilon": 0.00375,
  "learning_rate": 0.005,
  "training_steps": 10000,
  "per_epsilon": 0.05,
  "per_alpha": 0.8,
  "per_beta": 0.45,
  "minibatch_size": 128,
  "replay_buffer_size": 10000,
  "min_replay_buffer_size": 1250,
  "transfer_interval": 10,
  "n_step": 9,
  "kernel_initializer": "glorot_uniform",
  "loss_function": KLDivergence(), # could do categorical cross entropy 
  "clipnorm": 2.0,
  "discount_factor": 0.99,
  "atom_size": 81,
  "replay_interval": 4,
}
game_config = CartPoleConfig()
config = RainbowConfig(config_dict, game_config)
agent = RainbowAgent(env, config, name="Rainbow_CartPole-v1")

for param in agent.model.parameters():
  print(param)
print("start")
agent.train()

Using default save_intermediate_weights     : True
Using         training_steps                : 10000
Using         adam_epsilon                  : 0.00375
Using         learning_rate                 : 0.005
Using         clipnorm                      : 2.0
Using default optimizer                     : <class 'torch.optim.adam.Adam'>
Using         loss_function                 : <utils.utils.KLDivergence object at 0x1078ca4e0>
Using default activation                    : relu
Using         kernel_initializer            : glorot_uniform
Using         minibatch_size                : 128
Using         replay_buffer_size            : 10000
Using         min_replay_buffer_size        : 1250
Using default num_minibatches               : 1
Using default training_iterations           : 1
Using default conv_layers                   : []
Using         dense_layers_widths           : [128, 128]
Using         value_hidden_layers_widths    : [64, 64]
Using default advantage_hidden_layers_widths: 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gymnasium/envs/classic_control/cartpole.py:215: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(
/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/utils.py:269: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axs[row][col].legend()
/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/utils.py:199: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  axs[row][col].set_xlim(1, len(values))


score:  106.0
score:  113.0
score:  106.0
score:  130.0
score:  114.0


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gymnasium/envs/classic_control/cartpole.py:215: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(
/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/utils.py:269: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axs[row][col].legend()


score:  100.0
score:  116.0
score:  130.0
score:  115.0
score:  120.0


...

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gymnasium/envs/classic_control/cartpole.py:215: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(
/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/utils.py:269: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axs[row][col].legend()


score:  500.0
score:  500.0
score:  500.0
score:  500.0
score:  500.0


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gymnasium/envs/classic_control/cartpole.py:215: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(
/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/utils.py:269: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axs[row][col].legend()


score:  500.0
score:  500.0
score:  500.0
score:  500.0
score:  500.0


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gymnasium/envs/classic_control/cartpole.py:215: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(
/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/utils.py:269: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axs[row][col].legend()


## Rainbow on Classic Control
Hyperparameters come from revisitting rainbow

In [1]:
import gymnasium as gym
import sys

from utils import CategoricalCrossentropy, KLDivergence
sys.path.append('../..')
from dqn.rainbow.rainbow_agent import RainbowAgent
from agent_configs import RainbowConfig
from game_configs import ClassicControlConfig

# env = ClipReward(AtariPreprocessing(gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array"), terminal_on_life_loss=True), -1, 1) # as recommended by the original paper, should already include max pooling
# env = FrameStack(env, 4)


config_dict = {
  "dense_layers_widths": [512, 512],
  "value_hidden_layers_widths": [], # 
  "advatage_hidden_layers_widths": [], # 
  "adam_epsilon": 3.125e-4,
  "learning_rate": 0.001,
  "training_steps": 30000, #
  "per_epsilon": 1e-6, # 
  "per_alpha": 0.5, # 
  "per_beta": 0.4, # 
  "minibatch_size": 128,
  "replay_buffer_size": 50000,
  "min_replay_buffer_size": 500,
  "transfer_interval": 100,
  "n_step": 3,
  "kernel_initializer": "glorot_uniform", #
  "loss_function": KLDivergence(), # 
  "clipnorm": 2.0, # 
  "discount_factor": 0.99,
  "atom_size": 51,
  "replay_interval": 2,
}
game_config = ClassicControlConfig()
config = RainbowConfig(config_dict, game_config)

# game_config.v_min = 0
# game_config.v_max = 500
env = gym.make("CartPole-v1", render_mode="rgb_array")
agent = RainbowAgent(env, config, name="Rainbow_ClassicControl_CartPole-v1")
agent.train()

# game_config.v_min = -500
# game_config.v_max = 0
env = gym.make("Acrobot-v1", render_mode="rgb_array")
agent = RainbowAgent(env, config, name="Rainbow_ClassicControl_Acrobot-v1")
agent.train()

# game_config.v_min = -200
# game_config.v_max = 200
env = gym.make("LunarLander-v2", render_mode="rgb_array")
agent = RainbowAgent(env, config, name="Rainbow_ClassicControl_LunarLander-v2")
agent.train()

# game_config.v_min = -200
# game_config.v_max = -100
env = gym.make("MountainCar-v0", render_mode="rgb_array")
agent = RainbowAgent(env, config, name="Rainbow_ClassicControl_MountainCar-v0")
agent.train()

Using default save_intermediate_weights     : True
Using         training_steps                : 30000
Using         adam_epsilon                  : 0.0003125
Using         learning_rate                 : 0.001
Using         clipnorm                      : 2.0
Using default optimizer                     : <class 'torch.optim.adam.Adam'>
Using         loss_function                 : <utils.utils.KLDivergence object at 0x1779a2e40>
Using default activation                    : relu
Using         kernel_initializer            : glorot_uniform
Using         minibatch_size                : 128
Using         replay_buffer_size            : 50000
Using         min_replay_buffer_size        : 500
Using default num_minibatches               : 1
Using default training_iterations           : 1
Using default conv_layers                   : []
Using         dense_layers_widths           : [512, 512]
Using         value_hidden_layers_widths    : []
Using default advantage_hidden_layers_widths: []
Us

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/experiments/rainbow_and_ape-x folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


score:  -200.0
score:  -200.0
score:  -200.0
score:  -200.0
Moviepy - Building video checkpoints/Rainbow_ClassicControl_MountainCar-v0/step_1000/videos/Rainbow_ClassicControl_MountainCar-v0/1000/Rainbow_ClassicControl_MountainCar-v0-episode-4.mp4.
Moviepy - Writing video checkpoints/Rainbow_ClassicControl_MountainCar-v0/step_1000/videos/Rainbow_ClassicControl_MountainCar-v0/1000/Rainbow_ClassicControl_MountainCar-v0-episode-4.mp4



/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/utils.py:269: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axs[row][col].legend()
/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/utils.py:199: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  axs[row][col].set_xlim(1, len(values))


Moviepy - Done !
Moviepy - video ready checkpoints/Rainbow_ClassicControl_MountainCar-v0/step_1000/videos/Rainbow_ClassicControl_MountainCar-v0/1000/Rainbow_ClassicControl_MountainCar-v0-episode-4.mp4
score:  -200.0
score:  -200.0
score:  -200.0
score:  -200.0
score:  -200.0
Moviepy - Building video checkpoints/Rainbow_ClassicControl_MountainCar-v0/step_2000/videos/Rainbow_ClassicControl_MountainCar-v0/2000/Rainbow_ClassicControl_MountainCar-v0-episode-9.mp4.
Moviepy - Writing video checkpoints/Rainbow_ClassicControl_MountainCar-v0/step_2000/videos/Rainbow_ClassicControl_MountainCar-v0/2000/Rainbow_ClassicControl_MountainCar-v0-episode-9.mp4



...

Moviepy - Done !
Moviepy - video ready checkpoints/Rainbow_ClassicControl_MountainCar-v0/step_29000/videos/Rainbow_ClassicControl_MountainCar-v0/29000/Rainbow_ClassicControl_MountainCar-v0-episode-144.mp4
score:  -200.0


/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/utils.py:269: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axs[row][col].legend()


score:  -200.0
score:  -200.0
score:  -200.0
score:  -200.0
Moviepy - Building video checkpoints/Rainbow_ClassicControl_MountainCar-v0/step_29999/videos/Rainbow_ClassicControl_MountainCar-v0/29999/Rainbow_ClassicControl_MountainCar-v0-episode-149.mp4.
Moviepy - Writing video checkpoints/Rainbow_ClassicControl_MountainCar-v0/step_29999/videos/Rainbow_ClassicControl_MountainCar-v0/29999/Rainbow_ClassicControl_MountainCar-v0-episode-149.mp4



Moviepy - Done !
Moviepy - video ready checkpoints/Rainbow_ClassicControl_MountainCar-v0/step_29999/videos/Rainbow_ClassicControl_MountainCar-v0/29999/Rainbow_ClassicControl_MountainCar-v0-episode-149.mp4
score:  -200.0


/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/utils.py:269: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axs[row][col].legend()


## Rainbow on Atari